
# Forecast Statistical Analysis
## An assessment of Sea Surface Temperature (SST) RTOFS(forecast) and observed data in the Bay Area.
### This notebook presents an analysis of RTOFS predictions for a location where we have observed SST recorded.
### First, we load necessary packages and function. We prompt user date input for RTOFS. We have necessary functions for data extraction.
### Second, we fetch and process RTOFS (netCDF format) from an online server and processed using the xarray package. We fetch observed data. We group data by day and month. For RTOFS we also sort by lead day.
### Third, we merge both data sets and produce summary statistics and plot.

In [1]:
#looking at SST Forecast and Observed Data

from typing import Tuple, Dict, List

import numpy as np
import xarray as xr
import pandas as pd
import requests
import sys
import glob
import os

sys.path.append(os.path.relpath('../src'))
from statistical_analysis import statistical_analysis
from statistical_analysis import total_mean_bias



In [2]:
#user input for forecast dates

# format of lists [year,month,day]

#RTOFS
min_date_RTOFS = [2021,12,15] #today's date
max_date_RTOFS = [2021,12,20] #lead date ahead
LeadDay = 5 #choosing desired lead day

#need folder specifications

In [3]:
#downloading RTOFS Forecast data

def convert_time_tuple_to_str(x: Tuple[int, int, int]):
    """
    Converts a YYYY-MM-DD date, provided as a tuple, into a string

    :param x: tuple with year, month, day as integers
    :return: date string: YYYY-MM-DD, len=8, with 0s padding MM and DD as needed
    """
    return str(x[0]) + '-' + pad_with_zeros(x[1]) + '-' + pad_with_zeros(x[2])

def pad_with_zeros(x: int) -> str:
    return str(x) if x > 9 else '0' + str(x)

def get_rtofs2d(
        lon: float,
        lat: float,
        min_date: Tuple[int, int, int],
        max_date: Tuple[int, int, int],
        output_folder: str,
        use_si_units: str
) -> str:
    """
    Download RTOFS surface nowcast and forecast from ERDDAP

    :param min_lon: lower value of longitude for AOI bounding box
    :param max_lon: upper value of longitude for AOI bounding box
    :param min_lat: lower value of latitude for AOI bounding box
    :param max_lat: upper value of latitude for AOI bounding box
    :param min_date: current date (tuple with year, month, day)
    :param max_date: last day of forecast (tuple with year, month, day)
    :param output_folder: name of folder where downloaded netCDF file is stored
    :param use_si_units: keep original SI units?
    :return: path to new netCDF file created in device that holds the data
    """

    # Creating URL
    rtofs2d = {
        "dataset_id": "ncepRtofsG2DFore3hrlyProg_LonPM180",
        "environmental_variable": ["sst", "sss", "u_velocity", "v_velocity"]
    }
    #if lon < 74.16:
    #    lon += 360
    min_time = convert_time_tuple_to_str(min_date) + 'T00:00:00Z'
    max_time = convert_time_tuple_to_str(max_date) + 'T23:59:59Z'
    head = 'https://coastwatch.pfeg.noaa.gov/erddap/griddap/'
    body = (
            '[(' + min_time + '):1:(' + max_time + ')]' +
            '[(1.0):1:(1.0)]' +
            '[(' + str(lat) + '):1:(' + str(lat) + ')]' +
            '[(' + str(lon) + '):1:(' + str(lon) + ')]'
    )
    tail = ','.join([e + body for e in rtofs2d['environmental_variable']])
    url = head + rtofs2d['dataset_id'] + '.nc?' + tail

    # Fetching file
    #print(url)
    r = requests.get(url=url, allow_redirects=True)

    # Storing file
    out_file = output_folder + 'rtofs2d' + min_time +'.nc'
    tmp_file = output_folder + 'tmp.nc'
    open(tmp_file, 'wb').write(r.content)
    try:
        ds = xr.open_dataset(tmp_file)
    except Exception:
        print('RTOFS not available')
    else:
        if use_si_units == 'F':
            ds['sst'] = ((ds['sst'] * 9/5) + 32).assign_attrs({'units': 'F'})
            ds['u_velocity'] = (ds['u_velocity'] * 1.94384).assign_attrs({'units': 'knots'})
            ds['v_velocity'] = (ds['v_velocity'] * 1.94384).assign_attrs({'units': 'knots'})
        ds.to_netcdf(out_file)
    os.remove(tmp_file)

    return out_file


def extract_data_from_rtofs_nc(lon: float, lat: float, out_file: str, ocean_var: str) -> pd.DataFrame:
    """
    Read netCDF file from disk and extract data from one ocean variable
    :param lon: longitude of desired point
    :param lat: latitude of desired point
    :param nc_folder: location of 'rtofs2d.nc' file
    :param ocean_var: one of 'sst' (sea surface temperature),
                             'sss' (sea surface salinity),
                             'u_velocity' (surface zonal current speed),
                             'v_velocity' (surface meridional current speed)
    """
    if not ocean_var in ['sst', 'sss', 'u_velocity', 'v_velocity']:
        raise Exception('Variable not in file')
    b = xr.open_dataset(out_file)
    #if lon < 74.16:
    #    lon += 360
    lons = np.array(b['longitude'])
    lats = np.array(b['latitude'])
    lon_dists = list(np.abs(lons - lon))
    lat_dists = list(np.abs(lats - lat))
    idx_lon = lon_dists.index(min(lon_dists))
    idx_lat = lat_dists.index(min(lat_dists))
    nearest_lon = lons[idx_lon]
    nearest_lat = lats[idx_lat]
    if nearest_lon > 180:
        nearest_lon -= 360
    print('Data for (' + str(nearest_lon) + '; ' + str(nearest_lat) + ')')
    df = pd.DataFrame({'time': np.array(b['time']),
                       'lead_time': np.array(b['time'] - b['time'][0]),
                       ocean_var: np.array([x[0][idx_lat, idx_lon] for x in b[ocean_var]])})
    return df



In [4]:

folder = '/Users/amygizel/Documents/Seadog Internship/Model_Analysis/data/RTOFS_SFB/'

out_file=get_rtofs2d(lon=-122.6 - 0.0833,
            lat=37.8,
            min_date = (min_date_RTOFS[0],min_date_RTOFS[1],min_date_RTOFS[2]),
            max_date = (max_date_RTOFS[0],max_date_RTOFS[1],max_date_RTOFS[2]),
            output_folder=folder,
            use_si_units='T')

In [5]:

os.chdir(folder)
for out in glob.glob("rtofs*"):
    print(out)
big_df = pd.concat([extract_data_from_rtofs_nc(lon=-122.6 - 0.0833,
                                lat=37.8,
                                out_file=out_file,
                                ocean_var='sst') for out_file in glob.glob("rtofs*")])
big_df.sort_values('time')

rtofs2d2021-08-16T00:00:00Z.nc
rtofs2d2021-07-20T00:00:00Z.nc
rtofs2d2021-08-11T00:00:00Z.nc
rtofs2d2021-11-03T00:00:00Z.nc
rtofs2d2021-05-24T00:00:00Z.nc
rtofs2d2021-11-04T00:00:00Z.nc
rtofs2d2021-08-04T00:00:00Z.nc
rtofs2d2021-11-11T00:00:00Z.nc
rtofs2d2021-07-03T00:00:00Z.nc
rtofs2d2021-09-09T00:00:00Z.nc
rtofs2d2021-08-27T00:00:00Z.nc
rtofs2d2021-07-11T00:00:00Z.nc
rtofs2d2021-08-20T00:00:00Z.nc
rtofs2d2021-10-09T00:00:00Z.nc
rtofs2d2021-07-16T00:00:00Z.nc
rtofs2d2021-08-02T00:00:00Z.nc
rtofs2d2021-08-05T00:00:00Z.nc
rtofs2d2021-06-08T00:00:00Z.nc
rtofs2d2021-11-10T00:00:00Z.nc
rtofs2d2021-07-26T00:00:00Z.nc
rtofs2d2021-08-10T00:00:00Z.nc
rtofs2d2021-07-21T00:00:00Z.nc
rtofs2d2021-11-05T00:00:00Z.nc
rtofs2d2021-05-25T00:00:00Z.nc
rtofs2d2021-11-02T00:00:00Z.nc
rtofs2d2021-07-17T00:00:00Z.nc
rtofs2d2021-08-21T00:00:00Z.nc
rtofs2d2021-10-08T00:00:00Z.nc
rtofs2d2021-08-26T00:00:00Z.nc
rtofs2d2021-07-05T00:00:00Z.nc
rtofs2d2021-07-02T00:00:00Z.nc
rtofs2d2021-09-08T00:00:00Z.nc
rtofs2d2

,time,lead_time,sst
0,2021-05-20 00:00:00,0 days 00:00:00,9.004361
1,2021-05-20 03:00:00,0 days 03:00:00,8.724936
2,2021-05-20 06:00:00,0 days 06:00:00,8.462842
3,2021-05-20 09:00:00,0 days 09:00:00,8.296372
4,2021-05-20 12:00:00,0 days 12:00:00,8.189529
...,...,...,...
44,2021-12-20 12:00:00,5 days 12:00:00,NaN
45,2021-12-20 15:00:00,5 days 15:00:00,NaN
46,2021-12-20 18:00:00,5 days 18:00:00,NaN
47,2021-12-20 21:00:00,5 days 21:00:00,NaN


In [6]:
#cleaning RTOFS df and adding month, day and Lead Day
#taking average of RTOFS values throughout the day for specific lead days, month and day
#0,1,2,3,4,5 --> corresponds are Different lead days

big_df['month'] = [int(x) for x in big_df['time'].astype(str).str[-14:-12]]

big_df['day'] = [int(x) for x in big_df['time'].astype(str).str[-11:-8]]

big_df['Lead'] = [str(x) for x in big_df['lead_time'].astype(str).str[0:1]]

big_df= big_df.rename(columns={"sst": "RTOFS","lead_time":"Lead Day"})

big_df= big_df.dropna(subset = ["RTOFS"])

big_df= big_df.drop(['time'], axis=1)

big_df = big_df.groupby(by = ["day","month","Lead"]).mean()

big_df.columns = [' '.join(col) for col in big_df.columns]

big_df = big_df.reset_index(drop = False)

big_df = big_df.dropna(subset = ["R T O F S"])

big_df = big_df.rename(columns={"R T O F S": "RTOFS"})

big_df = big_df.sort_values(['month','day']).reset_index(drop = True)

big_df


,day,month,Lead,RTOFS
0,20,5,0,8.457231
1,21,5,1,8.835084
2,22,5,2,9.753041
3,23,5,3,10.146205
4,24,5,0,10.430880
...,...,...,...,...
786,17,11,4,14.217001
787,17,11,5,13.836976
788,18,11,4,14.064614
789,18,11,5,14.234098


In [7]:
#adding observed data

url = 'https://www.ndbc.noaa.gov/data/realtime2/46237.txt'

buoy = pd.read_csv(url, skiprows = [1], delim_whitespace=True)

buoy = buoy.rename(columns={"MM": "month","DD":"day"})

buoy = buoy.drop(columns = ["#YY","hh","mm","WDIR","WSPD","GST","WVHT","DPD","APD","MWD","PRES","ATMP","DEWP","VIS","PTDY","TIDE"])

buoy = buoy.groupby(by = ["day","month"]).mean()

buoy.columns = [' '.join(col) for col in buoy.columns]

buoy = buoy.reset_index(drop = False)

buoy = buoy.dropna(subset = ["W T M P"])

buoy = buoy.rename(columns={"W T M P": "Observed"})

buoy = buoy.sort_values(['month','day']).reset_index(drop = True)

buoy.head(5)


,day,month,Observed
0,1,11,14.854167
1,2,11,14.972917
2,3,11,15.061702
3,4,11,15.127660
4,5,11,15.160417


In [8]:
#merging Observed and Forecast Data
df = pd.merge(left=big_df, right=buoy[["Observed","day","month"]], on=["day","month"], how='inner')
df = df.sort_values(['month','day']).reset_index(drop = True)
df.head(20)

,day,month,Lead,RTOFS,Observed
0,1,11,0,14.447177,14.854167
1,1,11,1,14.405384,14.854167
2,1,11,2,14.762150,14.854167
3,1,11,3,14.446530,14.854167
4,1,11,4,14.130396,14.854167
5,2,11,0,14.081221,14.972917
6,2,11,1,14.241739,14.972917
7,2,11,2,14.144539,14.972917
8,2,11,3,14.627863,14.972917
9,2,11,4,14.300478,14.972917


In [9]:
df = [x[1] for x in df.groupby('Lead')]
df[LeadDay]

,day,month,Lead,RTOFS,Observed
15,3,11,5,14.613471,15.061702
21,4,11,5,14.853113,15.127660
28,5,11,5,14.267107,15.160417
35,6,11,5,13.754724,15.060417
42,7,11,5,13.642920,14.843478
49,8,11,5,13.022958,14.572340
55,9,11,5,13.086671,14.208333
61,10,11,5,13.526146,14.621277
67,11,11,5,13.543608,14.645833
73,12,11,5,14.231917,14.706250


In [10]:
statistical_analysis(LeadDay,df,"RTOFS","Observed",1)

0.6879427224218452 is the mean bias for lead time of 5 days
0.8375103086974762 is the root mean squared error for lead time of 5 days
0.375 is the percent gross error for lead time of 5 days
4.67749896621485 is the mean absolute percentage error 5 days


In [11]:
total_mean_bias([1,2,3,4,5,6],df,"RTOFS","Observed")

0.7049791653820443